In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
os.environ['OPENBLAS_NUM_THREADS'] = '1'
sys.path.insert(0,os.path.expandvars('$CODE_MEMORY_ERRORS'))

import config2
ipy = get_ipython()
#data_dir_general = '/home/demitau/data_Quentin'00
#data_subdir_mem_err_main = 'full_experiments/data2'
from os.path import join as pjoin
import pandas as pd
from config2 import *
import mne
import seaborn as sns

print('pandas ver ', pd.__version__)
print('seaborn ver ', sns.__version__)
print('MNE ver ', mne.__version__)
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#data_dir_general = '/home/demitau/data_Quentin'
#data_subdir_mem_err_main = 'full_experiments/data2'
#data_dir_input = pjoin(data_dir_general,data_subdir_mem_err_main)
#scripts_dir = pjoin(data_dir_general,'full_experiments','scripts2')

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
#scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'previous_analyses')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'), 'figure')

from dateutil import parser
from error_sensitivity import getAnalysisVarnames

print(data_dir_input, scripts_dir)

subjects = [f for f in os.listdir(data_dir_input) if f.startswith('sub') ]
subjects = list(sorted(subjects))
print(subjects)

import figure.plots


$$e_n = \mathrm{observation_n} =\\= \mathrm{endpoint}_n- \mathrm{target\_angle}_n =\\
=x_n + \mathrm{perturbation}_n$$

\\

$$x_{n+1} = \alpha \cdot x_{n} + \beta_n \cdot e_n$$




* Empirical: $\beta_n = \displaystyle\frac{e_n - e_{n-1}}{|e_{n-1}|}$

* Dierichsen:     $\beta_n \equiv \beta$

* Tan: $\beta_n = c \cdot \displaystyle\frac{\mu^2}{ \sigma^2 }$

* Herzfeld:     $ \beta_n = \beta_{n-1} + \gamma\cdot \mathrm{sign}(e_{n-2} \cdot e_{n-1})  $

# Plot sliding

In [ ]:
gc.collect()

In [ ]:
# collect
import gc; gc.collect()
plot_diff_vs_es = 0
plot_diff = 0

regression_type = 'Ridge'
time_lockeds = ['target', 'feedback']
#freq_names = ['broad']
#freq_names = ['broad', 'beta', 'gamma']
freq_names = ['broad', 'beta', 'gamma']
control_types = [ 'movement']
#control_types = [ 'movement', 'feedback']
#subject_inds = [15,16,17,18,19] # up to 10 fine
subject_inds = np.arange(len(subjects))
#subject_inds = list(range(12)) + list(range(13,20))
#subject_inds = [12]

# good combination to plot robust = 1, discard twice = 0,1
#time_start = parser.parse('00:00 of 2nd june 2023'); 
#time_end = parser.parse('16:03 of 3rd june 2023'); 

#time_start = parser.parse('18:00 of 20th june 2023');  # 1st, for abstract
time_start = parser.parse('13:48 of 21st june 2023'); 
#time_end = parser.parse('16:03 of 3rd june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 

custom_prefix = None

use_preload_df = 0
load_df        = 0
exit_after = 'collect'
#exit_after = 'end'
plot_scores_per_subj = 0
ktes = ['err_sens','err_sens_Tan','err_sens_Herz',
        'error', 'error_pred_Tan','error_pred_Herz',
        'error_pred_Died', 
        'prev_error',
        'trials','trialwe',
        'trialwpertstage_wb',
        'trialwtgt','trialwtgt_wpertstage_wb',
        'RT','target_inds',
       'movement_duration']
#b2bdef = 'err_sens,prev_error,error'
b2bdef = ['err_sens,prev_error,error',
         'err_sens_Tan,prev_error,error',
         'err_sens_Herz,prev_error,error',]
varnames = ktes
tmin_desired = None
require_all_subjects = 0

import traceback
DEBUG = 0
save_df = 0

#%debug
failed = []
dfs = []
for hpass in ['no_filter']: #,'0.1']:
    for env in [['stable','random']]:
        #for freq_name in list(freq_name2freq.keys() ):
        for freq_name in freq_names:
            for time_locked in time_lockeds:
                for control_type in control_types:                    
                    df = None
                    s=  ','.join( getAnalysisVarnames(time_locked, control_type)[0] )
                    varnames_b2b = [s, 'err_sens,prev_error,error']

                    output_folder = f'corr_spoc_es_sliding2_{hpass}'        
                    script_name = pjoin(scripts_dir,
                            'correlation_error_sensitivity_spoc_decoding_sliding.py')
                    try:
                        ipy.run_line_magic('run','-i {script_name}')                  
                        dfs += [df]                        
#                         if not DEBUG:
#                             del df
                    except Exception as e:                    
                        print('EXC',e)
                        exc_info = sys.exc_info()
                        exc = traceback.TracebackException(*exc_info, capture_locals=True)
                        ei = exc_info[2]    
                        #display(ei.tb_frame)
                        psf = ei

                        lfprev = None; lf = None
                        lfs = []
                        di = 0
                        while psf.tb_frame is not None:
                            stackframe = psf.tb_frame
                            display(di, stackframe)

                            psf = psf.tb_next  
                            if 'conda' not in stackframe.f_code.co_filename:        
                                lfprev = lf
                                lf = stackframe.f_locals
                                lfs += [lf]
                                di += 1
                            if psf is None:
                                break    

                        #raise e

                        print('Exc for ', hpass,freq_name)
                        failed += [ (hpass, freq_name)]
                        print('EXC',e)
                    #df.to_pickle( pjoin(path_fig, save_folder, 'df') )                                        
                    import gc; gc.collect()
                    plt.close()
            #break
        #break
    #break
print('Collection finished')

dfc =pd.concat(dfs)

dfc_allcollected = dfc

catcols = ['subject', 'fn', 'fn_full', 'SLURM_job_id', 'env',
       'trial_group_col_calc', 'rt', 'time_locked', 'control_type',
       'custom_suffix', 'freq_name', 'tmin', 'tmax', 'dec_type']
for col in catcols:
    dfc[col] = dfc[col].astype('category')

dfc['jobind'] = dfc['SLURM_job_id'].apply(lambda x: int(x.split('_')[1] ) )

In [ ]:
df1.columns

In [ ]:
from postproc import collectResultsPrefilled,collectResults0
tmin_desired = None
hpass = 'no_filter'
time_start = parser.parse('13:48 of 21st june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 
time_lockeds = ['target', 'feedback']
#freq_names = ['broad']
freq_names = ['broad', 'beta', 'gamma']
control_types = [ 'movement']

output_folder = f'corr_spoc_es_sliding2_{hpass}'        
wildcard = "*CNslideModelES*_sh0.250*.npz"
df1 = collectResults0(subjects,output_folder, wildcard=wildcard)
print('len(df1)', len(df1))
assert len(df1)

In [ ]:
print('tmin_desired = ',tmin_desired)

qs = ''
#qs += 'time_locked.isin(@time_lockeds) and control_type.isin(@control_types)'
#qs += ' freq_name.isin(@freq_names)'
qs += ' mtime >= @time_start'
qs += ' and mtime <= @time_end'
#qs += ' and custom_suffix.str.endswith("_sh0.232")'
qs += ' and custom_suffix.str.contains("c1_")'
if tmin_desired is not None:
    if isinstance(tmin_desired,list):
        qs += ' and tmin.isin(@tmin_desired)'
    else:
        qs += ' and tmin == @tmin_desired'
df2 = df1.query(qs).copy()
print('after restricting to time interval ',len(df1),len(df2))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df2['custom_suffix'].unique()

In [ ]:
df2.groupby('custom_suffix').size()

In [ ]:
df2.groupby('custom_suffix')['mtime'].mean()

In [ ]:
plt.hist(df2['mtime'])

In [ ]:
df2.iloc[0]['fn_full']

In [ ]:
/p/project/icei-hbp-2022-0017

In [ ]:
#df2['fn_full_scratch'] = df2['fn_full'].apply('')

In [ ]:
dirs = df2.sort_values('mtime').groupby('subject').\
    agg(['last'])['fn_full']['last'].apply(lambda x: os.path.split(x)[0])
dirs.iloc[0]

In [ ]:
path_data

In [ ]:
'/p/scratch/icei-hbp-2022-0017/memerr'

In [ ]:
df2['fn_full_scratch'] = None

import shutil
scratch_copy = True
copied_to_scratch = False
if scratch_copy:
    for diri, subdir_cur in enumerate(dirs):
        subdir_cur_tmp = subdir_cur.replace(path_data, 
                pjoin( os.path.expandvars('$SCRATCH'), 'memerr', 'data2' ))
        print(diri, subdir_cur, '  -->  ', subdir_cur_tmp)
        #continue
        import subprocess as sp
        if not os.path.exists(subdir_cur_tmp):
            os.makedirs(subdir_cur_tmp)
        #out = sp.getoutput(f'cp -r {pjoin(gv.data_dir,subdir_cur)}/_!_* {pjoin(gv.data_dir_tmp,subdir_cur)}')
        
        s = f'rsync -tv {subdir_cur}/{wildcard} {subdir_cur_tmp}/'
        print(s)
        out = sp.getoutput(s)
        #shutil.copytree(pjoin(gv.data_dir,subdir_cur), 
        #                pjoin(gv.data_dir_tmp,subdir_cur) )
        print(out)
    copied_to_scratch = True
    df2['fn_full_scratch'] = df2['fn_full'].str.replace(path_data,  pjoin( os.path.expandvars('$SCRATCH'), 'memerr', 'data2' ) )
else:
    print('Skipping scratch copy')

In [ ]:
df2['fn_full_scratch'] = df2['fn_full'].str.replace(path_data,  
        pjoin( os.path.expandvars('$SCRATCH'), 'memerr', 'data2' ) )

In [ ]:
df2

In [ ]:
import gc; gc.collect()

In [ ]:
# slow
#%debug
dfc = collectResultsPrefilled(df2)
dfc_allcollected = dfc

catcols = ['subject', 'fn', 'fn_full', 'SLURM_job_id', 'env',
       'trial_group_col_calc', 'rt', 'time_locked', 'control_type',
       'custom_suffix', 'freq_name', 'tmin', 'tmax', 'dec_type']
for col in catcols:
    dfc[col] = dfc[col].astype('category')

dfc['jobind'] = dfc['SLURM_job_id'].apply(lambda x: int(x.split('_')[1] ) )
print( len(dfc) )

In [ ]:
dfc[['mtime','rt','custom_suffix']]

In [ ]:
#df3['b2b_varset']

In [ ]:
#trialwpertstage_wb_non_hit, ValueError: Incompatible indexer with Series

In [ ]:
print(len(dfc))
print(dfc['subject'].unique())
suffixes_present = dfc['custom_suffix'].unique()
print(list(suffixes_present))
dfc_types_present = dfc['dec_type'].unique(); 
print(list(dfc_types_present))

In [ ]:
dfc.duplicated(catcols).sum()

In [ ]:
list(suffixes_present)

In [ ]:
print(dfc['jobind'].astype('int').max())

In [ ]:
  #won't work if columns contain composite types
#dfc.duplicated()

# print( dfc['time_locked'].unique() )
# for dfi,df in enumerate(dfs):
#     print(dfi)
#     if df is None:
#         continue
#     print(df.duplicated(catcols).sum() , len(df), df['time_locked'].unique() )

In [ ]:
dfc[catcols].describe(include='all').loc['unique', :]

#### dbg

In [ ]:
tmin_desired

In [ ]:
time_lockeds

In [ ]:
time_locked

In [ ]:
try:
    collectDecodingResults(subjects, subject_inds[:1], output_folder,freq_name, 
                    env, control_type, 
                    time_locked, custom_prefix,                       
                     tmin_desired,                                                                                                                             
                     varnames, varnames_b2b,                                                                                                                   
                     time_start, time_end, verbose=1) 
    
except Exception as e:                    
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]    
    #display(ei.tb_frame)
    psf = ei

    lfprev = None; lf = None
    lfs = []
    di = 0
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame        

        psf = psf.tb_next  
        if ('conda' not in stackframe.f_code.co_filename) and\
            ('mamba' not in stackframe.f_code.co_filename):        
            display(di, stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            di += 1
        if psf is None:
            break 
        
    raise e

In [ ]:
lf['order']

In [ ]:
type( lf['order'] )

In [ ]:
lf['df_to_set']['prev_movement,prev_error,error,prev_belief_varnames']

In [ ]:
 lf['kvv'] ,  lf['index'] , lf['key']#, lf['df_to_set'][lf['index'] , lf['key']]

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [1, 2], 'B': [3, 4]})
df.at[0, 'A'] = list(np.array([5, 6]))

In [ ]:
Length of values (315) does not match length of index (264)

In [ ]:
lf['df_to_set'][lf['index'] , lf['key']] = lf['kvv'].tolist()

In [ ]:
varnames_b2b

#### check collected

In [ ]:
dec_types_found = dfc_allcollected['dec_type'].unique()
print('dec_types_found = ',dec_types_found)
assert len(dec_types_found) > 1

In [ ]:
def calcTminSetVariability(df_, cols):
    r = dfc.groupby(cols)['tmin'].nunique()#.apply(lambda x: len(x) )
    return r.to_frame().describe().loc['std'].values[0]

In [ ]:
#dfc.groupby(cols_unique).size().max()

In [ ]:
import pandas as pd

# Define a function to find the minimal set of categorical columns
def find_minimal_columns(dfc):
  # Get the list of categorical columns in the DataFrame
  cat_cols = dfc.select_dtypes(include='category').columns.tolist()
  # Initialize an empty list to store the selected columns
  selected_cols = []
  # Initialize a variable to store the number of duplicates
  num_duplicates = dfc.duplicated().sum()
  # Loop through the categorical columns
  for col in cat_cols:
    # Add the current column to the selected columns
    selected_cols.append(col)
    # Create a new DataFrame with only the selected columns
    dfc_new = dfc[selected_cols]
    # Calculate the number of duplicates in the new DataFrame
    num_duplicates_new = dfc_new.duplicated().sum()
    # If the number of duplicates is zero, break the loop
    if num_duplicates_new == 0:
        break
    # Otherwise, update the number of duplicates
    else:
        num_duplicates = num_duplicates_new
  # Return the selected columns
  return selected_cols

# Call the function on dfc and print the result
minimal_columns = find_minimal_columns(dfc)
print('The minimal set of categorical columns is:', minimal_columns)

In [ ]:
#dfc.describe?

In [ ]:
# make it hang
#dfc.describe(include='all').loc['unique', :]

In [ ]:
#dfc.columns

In [ ]:
print(dfc['mtime'].min(), dfc['mtime'].max())

cols_unique = ['custom_suffix', 'trial_group_col_calc', 
    'time_locked','control_type', 
    'freq_name', 'dec_type', 'env','subject','tmin']
if dfc.duplicated(cols_unique).any():
    print('DUPLICATES:')
    display( dfc[ dfc.duplicated(cols_unique) ] )
else:
    print('No duplicates (tmin + custom_prefix)!')

print( len(dfc), dfc['subject'].nunique(), dfc['control_type'].nunique() )
#      subject_inds )


cols_unique_ = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'tmin', 'dec_type']
if dfc.duplicated(cols_unique_).any():
    sz =  dfc.groupby(cols_unique_).size()
    print('Within suffix (fixed tmin): min dup = {}, max dup = {}'.\
                format(sz.min(), sz.max() ) )
else:
    print('No duplicates for same time across suffixes!')
    
cols_unique_2 = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'dec_type', 'custom_suffix']
sz =  dfc.groupby(cols_unique_2).size()
# if std of len is > 0, then we might need be in situation where there were two calc runs 
# with different tmins set
print('Within tmin (fixed suffix): min dup = {}, max dup = {}. Std of len = {:.2f}'.\
            format(sz.min(), sz.max(),  calcTminSetVariability(dfc, cols_unique_2) ) )
    
grp = dfc.groupby(['custom_suffix','time_locked'])
print('  Times within suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) )

suffixes_present = dfc['custom_suffix'].unique()
print(suffixes_present)

In [ ]:
dfc_uncleaned = dfc

In [ ]:
suffs = [f'CNslide_trim0_dhittw1_s_scX0Y2c0_drall_dtall_{freq}_sh0.232' for freq in ['broad','beta','gamma']]
suffs

In [ ]:
dfc = dfc_uncleaned.query('custom_suffix in @suffs')

In [ ]:
suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall_broad_sh0.232'
if suff not in suffixes_present:
    suff = suffixes_present[0]
tmins_good = dfc.query('custom_suffix == @suff')['tmin'].unique()
std = np.diff( tmins_good.astype(float) ).std() 
print(std)
assert std < 1e-2 # recal we round to two digits
dfc_clean2 = dfc.query('tmin in @tmins_good')
display( dfc_clean2.groupby('custom_suffix')['mtime'].describe(datetime_is_numeric=True) )

In [ ]:
#dfc_ = dfc.query('dec_type == "classic"')

In [ ]:
#dfc = dfc.query('custom_suffix == @suff')

In [ ]:
dfc = dfc_clean2

### multi calcs mgmt

In [ ]:

#reset_index()['tmin'].apply()

In [ ]:
sz.reset_index()[0].unique()

In [ ]:
# all
grp['mtime'].describe(datetime_is_numeric=True)['max']

In [ ]:
dfc_allcollected['mtime'].dt?

In [ ]:
dfc_allcollected['mtime_sec'] = dfc_allcollected['mtime'].values.astype(np.int64) // 10 ** 9

In [ ]:
cols_unique_

In [ ]:
cols_unique_

In [ ]:
dfc_allcollected_.reset_index(drop=True)

In [ ]:
dfc_allcollected_ = dfc_allcollected.sort_values('mtime_sec').reset_index(drop=True)
grp = dfc_allcollected_.groupby(cols_unique_)
sz =  grp.size()
print('Within suffix: min dup = {}, max dup = {}'.\
            format(sz.min(), sz.max() ) )
#grp = dfc_allcollected_.groupby(cols_unique_2)
grp = dfc_allcollected_.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True)['max'] )

In [ ]:
#dfc_allcollected_.groupby(cols_unique_2).size()

In [ ]:
grp = dfc_allcollected_.groupby(cols_unique_)
idx = grp['mtime'].idxmax()

In [ ]:
len(idx), len(dfc_allcollected_)

In [ ]:
grp['mtime'].max?

In [ ]:
grp.max('mtime').reset_index()#.query('custom_suffix == "CNslide_trim0_dhittw0_s_scX1Y1_drall_dtall"')

In [ ]:
grp.max('mtime').reset_index().sort_values('mtime')['mtime']

In [ ]:
dfc_allcollected_.loc[idx].sort_values('mtime')['mtime']

In [ ]:
# select max time within suffix
dfc_clean = aggRows(dfc_allcollected_, 'mtime_sec', 
                   operation= 'max', coltake='corresp', colgrp = cols_unique_)
grp = dfc_clean.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) ['max'] )
#dfc = dfc_clean

In [ ]:
dfc_clean = dfc_allcollected_.sort_values('mtime', ascending=False).drop_duplicates(cols_unique_)
grp = dfc_clean.groupby('custom_suffix')
display( grp['mtime'].describe(datetime_is_numeric=True) ['max'] )

In [ ]:
dftmp0 = dfc_allcollected_.sample(1000).sort_values('mtime')

In [ ]:
dftmp1 = dftmp 

In [ ]:
dftmp2 = dftmp 

In [ ]:
dftmp3 = dftmp 

In [ ]:
# select max time within suffix
difsec = 0
while difsec < 1e-10:
    dftmp = dftmp3.sample(5).sort_values('mtime')
    #dftmp = dftmp1.sample(20).sort_values('mtime')
    #dftmp = dftmp0.sample(100).sort_values('mtime')
    #dftmp = dftmp0
    grp = dftmp.groupby('custom_suffix')
    desc_max0 = grp['mtime'].describe(datetime_is_numeric=True) ['max']
    display(desc_max0  )
    dfc_clean = aggRows(dftmp, 'mtime_sec', 
                       operation= 'max', coltake='corresp', colgrp = cols_unique_)
    grp = dfc_clean.groupby('custom_suffix')
    desc_max = grp['mtime'].describe(datetime_is_numeric=True) ['max']
    display(desc_max  )
    #dfc = dfc_clean
    difsec = (desc_max0.iloc[0] - desc_max.iloc[0]).total_seconds()
    print(difsec)

In [ ]:
grp0 = dftmp.groupby(cols_unique_)
grp0['mtime'].idxmax()

In [ ]:
cols_unique_

In [ ]:
grp0['mtime'].max().reset_index()

In [ ]:
set(cols_unique) ^ set(cols_unique_)

In [ ]:
cols = ['subject',
 'env',
 'tmin',
 'dec_type',
 'custom_suffix', 'mtime']

In [ ]:
dftmp[cols].sort_values('mtime')

In [ ]:
# remove 4164 -- compared to 5172 same everything except custom suffix
dfc_clean[cols].sort_values('mtime')

In [ ]:
dfc = dfc_allcollected

In [ ]:
from behav_proc import aggRowsws

In [ ]:
from datetime import timedelta

In [ ]:
tdm.total_seconds()

In [ ]:
tdm = dfc['mtime'].diff().min()

In [ ]:
isinstance( date_cutoff, datetime, )

In [ ]:
len(dfc_clean), len(dfc)

In [ ]:
#dfc.groupby(cols_unique_)

In [ ]:
date_cutoff = parser.parse('02 june 2023 15:00')
#date_cutoff = parser.parse('02 june 2023 15:00')
dfc_cut = dfc[dfc['mtime'] > date_cutoff]

In [ ]:
dfc['time_locked'].unique()

In [ ]:
from postproc import extractB2Bcols

In [ ]:
dfc = dfc.query('custom_suffix.str.startswith("CNslide")'
                ' and control_type == "movement" and time_locked == "target"')
dfc_, df_b2b_plot = extractB2Bcols(dfc)

sz =  dfc.groupby(cols_unique_).size()
print('min dup = {}, max dup = {}'.\
            format(sz.min(), sz.max() ) )

In [ ]:
#hours_since

In [ ]:
np.max(days_since)

In [ ]:
grp.groups.keys()

### hist of times

In [ ]:
sns.set(font_scale=1.)
import matplotlib.dates as mdates
from datetime import datetime, timedelta
grp = dfc.groupby(['custom_suffix'] )
for suff,inds in grp.groups.items():
    df_ = dfc.loc[inds]
    date_list = list( sorted( df_['mtime'] ) )
    #date_st = datetime(2023,5,31)
    date_st = datetime(2023,6,2)
    # Calculate the number of days since May 31st and the number of minutes since the corresponding day start
    days_since = [(date - date_st).days for date in date_list]
    hours_since = [(date - date_st).components.hours for date in date_list]
    minutes_since_ds = [(date - date.replace(hour=0, minute=0)).total_seconds() / 60 for date in date_list]
    minutes_since = [(date - date_st).total_seconds() / 60 for date in date_list]

    # Create a histogram of the dates
    fig = plt.figure(figsize=(10,3))
    ax = plt.gca()
    ax.hist(minutes_since, bins=30)

    # Set the x-axis tick format
    ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
    dts = [date_st + timedelta(days=int(day), minutes=int(minute)) for day,minute in zip(days_since, minutes_since)]
    xts = [f'{int(day)}\n{int(minute)}\n{date.strftime("%H:%M")}' for day,minute,date in zip(days_since, minutes_since_ds,dts)]
    #ax.set_xticks(range(len(days_since)))
    
    def ff(x,f):
        x = int(x)
        #print(x,f)
        r = str(x)
        if x < len(days_since):
            r = f'{int(days_since[(x)])}\n{int(minutes_since_ds[(x)])}\n{date_list[(x)].strftime("%H:%M")}'
        return r
    ax.xaxis.set_major_formatter(ff)
    
    #ax.set_xticklabels(xts)

    # Set the x-axis label and title
    ax.set_xlabel(f'Days Since {date_st} and Minutes Since Day Start')
    ax.set_ylabel('Frequency')
    ax.set_title('Histogram of Dates ' + suff)
    
    #break

    #plt.show()

In [ ]:
len(days_since)

In [ ]:
tmins_all = dfc.tmin.unique()
print(tmins_all), len(tmins_all)

In [ ]:
#tmins_to_plot = ['-0.36', '-0.13', '0.10', '0.34']
tmins_to_plot = ['-1.06', '-0.82', '-0.59', '-0.36', '-0.13', '0.10', '0.34', '0.57', '0.80', '1.03']
tmins_to_plot = tmins_all

In [ ]:
#dfc.columns

#### reshape debug

In [ ]:
display(arr_reshaped)

In [ ]:
arr.reshape(8, 3) - arr.transpose(0,2,1).reshape(8, 3)

In [ ]:
arr_reshaped.shape, arr.transpose(0,2,1).shape

In [ ]:
arr_back[:,0,:], arr_back[:,1,:]

In [ ]:
arr_back[0,:,:],arr_back[1,:,:]

In [ ]:
import numpy as np

arr = np.arange(24).reshape(2, 3, 4)
arr[:,1,:] *= 10
arr[:,2,:] *= 100
print("Original array:\n", arr)

# Reshape to 3x8 array
arr_reshaped = arr.transpose(0,2,1).reshape(-1, 3)
print("\nReshaped array:\n", arr_reshaped)
from sklearn.preprocessing import RobustScaler as robsc
arr_reshaped1 = robsc(with_centering=True, unit_variance=False).fit_transform(arr_reshaped)

print(arr_reshaped1)

# Reshape back to 2x3x4 array
arr_back = arr_reshaped1.reshape(arr.transpose(0,2,1).shape).transpose((0,2,1))# * 100
print("\nBack to original shape:\n", arr_back)


In [ ]:
tmins_to_plot

In [ ]:
suffixes_present

## continue

In [ ]:
varsets = dfc.query('dec_type == "classic"')['varset'].unique()
print(list(varsets))
varnames0 = list(set(';'.join(varsets).split(';') ) )#.split(',')
print('varnames0',varnames0)

In [ ]:
from postproc import extractClassicCols
#varnames0 = ['prev_movement','prev_belief','movement','belief','feedback',
#             'error','prev_error','err_sens','next_error','correction' ]
#varnames0 = varnames
df_class_plot = extractClassicCols(dfc, defvarcolns=varnames0,
                                  error_handling = 'ignore')

In [ ]:
from postproc import extractB2Bcols
try:
    dfc_, df_b2b_plot0 = extractB2Bcols(dfc, toclean=varnames0)
except Exception as e:                    
    print('EXC',e)
    exc_info = sys.exc_info()
    exc = traceback.TracebackException(*exc_info, capture_locals=True)
    ei = exc_info[2]    
    #display(ei.tb_frame)
    psf = ei

    lfprev = None; lf = None
    lfs = []
    di = 0
    while psf.tb_frame is not None:
        stackframe = psf.tb_frame        

        psf = psf.tb_next  
        if ('conda' not in stackframe.f_code.co_filename) and\
            ('mamba' not in stackframe.f_code.co_filename):        
            display(di, stackframe)
            lfprev = lf
            lf = stackframe.f_locals
            lfs += [lf]
            di += 1
        if psf is None:
            break    

    #raise e
    print('EXC',e)

In [ ]:
[cn for cn in list(df_b2b_plot0.columns) if cn.find('err_sens') >= 0]

In [ ]:
df_b2b_plot = pd.concat([df_class_plot,df_b2b_plot0] )

In [ ]:
df_b2b_plot

In [ ]:
#df_b2b_plot = df_class_plot

In [ ]:
cols0 = ['rt','custom_suffix','time_locked','control_type','dec_type',
         'varseti','varname','tmin']
cols_ = list( set(cols_unique) - set(['env']) ); cols_

In [ ]:
def f(row):
    x = row['vals_to_plot']
    from postproc import ListWrap
    if isinstance(x, ListWrap):
        x = x.data    
    else:
        if not isinstance(x,float):
            print(type(x))
    if isinstance(x, list) or isinstance(x, np.ndarray):        
        if len(x ) != 1:
            print(row['dec_type'],row)
            raise ValueError('ee')
        x = x[0]
    return x
df_b2b_plot['vals_to_plot'] = df_b2b_plot.apply(f,1)

rnd = df_b2b_plot.query('env == "random"').set_index(['subject'] + cols0)
stb = df_b2b_plot.query('env == "stable"').set_index(['subject'] + cols0)

dfconddif = stb['vals_to_plot'] - rnd['vals_to_plot']
dfconddif = dfconddif.to_frame().rename(columns={'vals_to_plot':'difcond'})

In [ ]:
#get tmins
tl2tmins_to_plot = {}
tl2indhome = {}
for tl in df_b2b_plot['time_locked'].unique():    
    tmins_to_plot  = list(sorted( df_b2b_plot.query('time_locked == @tl')['tmin'].unique(), key=lambda x: float(x) ) )
    tl2tmins_to_plot[tl] = tmins_to_plot
    print(tl, len(tmins_to_plot))
    
    if '-0.50' in tmins_to_plot:
        indhome = tmins_to_plot.index('-0.50'); print(indhome)
    elif '-0.59' in tmins_to_plot:
        indhome = tmins_to_plot.index('-0.59'); print(indhome)
    else:
        try:
            indhome = tmins_to_plot.index('0.09'); print(indhome)
        except ValueError as e:
            indhome = None
    
    tl2indhome[tl] = indhome
print( tl2indhome )

control_type_present = df_b2b_plot['control_type'].unique(); 
print(control_type_present)
time_lockeds_present = df_b2b_plot['time_locked'].unique(); control_type_present
print(time_lockeds_present)

In [ ]:
df_b2b_plot.groupby(['dec_type','varseti','varname']).size()

In [ ]:
grp_stb_sz = stb.groupby(cols0).size()
if grp_stb_sz.max() > 20:
#display(grp_stb_sz)
    row = grp_stb_sz[grp_stb_sz > 20].to_frame().iloc[0]
    inds = stb.groupby(cols0).groups[row.name]

#stb.groupby(['rt']).size()

    display(stb.loc[inds])

In [ ]:
#dfconddif

In [ ]:
# cols_ = list( set(cols_unique) - set(['env']) ); cols_
# # takes several minutes
# grp = df_b2b_plot.groupby(cols_ + ['varseti','varname'])
# assert grp.size().max() == 2

# def f(dftmp):
#     assert set( dftmp['env'].unique() ) == set(['random','stable'])
#     d = dftmp.set_index('env').to_dict('index')
#     r = d['stable']['vals_to_plot'] - d['random']['vals_to_plot']
#     return r
# dfconddif = grp.apply(f)
# dfconddif= dfconddif.reset_index(name='difcond')

In [ ]:
#display(grp.size().describe())

In [ ]:
# todo show slurm id

In [ ]:
# compare mean vs random decoding
from base2 import decod_stats # X - participant x times
def f(dftmp):
    r = dftmp.pivot(index='subject', columns='tmin', values='difcond')
    time_order = r.columns.tolist()
    #print(r.shape)  #participant x times
    r = r.to_numpy()
    pvals =  decod_stats(r, n_jobs=4, n_permutations = 2**10)
    res = pd.DataFrame({'pvalue': pvals, 'tmin':time_order})#.set_index('tmin')
    #display(res)
    return res
#cols0 = ['custom_suffix','time_locked','control_type','dec_type','varseti','varname','tmin']
grp = dfconddif.groupby(cols0)
assert grp.size().max() == 20,  grp.size().max()
dfconddif_pv = dfconddif.reset_index().groupby(list(set(cols0) - set(['tmin']))).apply(f)
dfconddif_pv['signif'] = dfconddif_pv['pvalue'] < 0.05

#base2.py:82: RuntimeWarning: Provided stat_fun does not treat variables independently. Setting buffer_size to None.
#  T_obs_, clusters, p_values, _ = permutation_cluster_1samp_test(
#/p/project/icei-hbp-2022-0017/memerr/code/base2.py:82: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv

In [ ]:
# compare stable dec with zero
from base2 import decod_stats # X - participant x times
def f(dftmp):
    r = dftmp.pivot(index='subject', columns='tmin', values='vals_to_plot')
    time_order = r.columns.tolist()
    #print(r.shape)  #participant x times
    r = r.to_numpy()
    pvals =  decod_stats(r, n_jobs=4, n_permutations = 2**10)
    res = pd.DataFrame({'pvalue': pvals, 'tmin':time_order})#.set_index('tmin')
    #display(res)
    return res

#cols_ = list( set(cols_unique) - set(['subject'])); #cols_
#cols__ = cols_ + ['varseti','varname']
#print(cols__)
df_ = df_b2b_plot.query('env == "stable"')
#grp = df_.groupby(cols__)
grp = df_.groupby(cols0)
#grp = dfconddif.groupby(['custom_suffix','varseti','varname','control_type','tmin'])
assert grp.size().max() == 20, grp.size().max()
dfconddif0_pv = df_.groupby(list(set(cols0) - set(['tmin']))).apply(f)
dfconddif0_pv['signif'] = dfconddif0_pv['pvalue'] < 0.05

# compare random dec with zero
df_ = df_b2b_plot.query('env == "random"')
#grp = df_.groupby(cols__)
grp = df_.groupby(cols0)
#grp = dfconddif.groupby(['custom_suffix','varseti','varname','control_type','tmin'])
assert grp.size().max() == 20, grp.size().max()
dfconddif0r_pv = df_.groupby(list(set(cols0) - set(['tmin']))).apply(f)
dfconddif0r_pv['signif'] = dfconddif0r_pv['pvalue'] < 0.05

In [ ]:
#cols_unique

In [ ]:
gc.collect()

In [ ]:
#suffixes_present

In [ ]:
#df_b2b_plot.columns

In [ ]:
cols0

In [ ]:
#df_b2b_plot.groupby(['subject'] + cols0 + ['env']).size()

In [ ]:
#df_b2b_plot.groupby(cols0).size()

In [ ]:
df_b2b_plot.groupby(cols0)['subject'].nunique().max(),\
    df_b2b_plot.groupby(cols0)['subject'].nunique().min()

In [ ]:
gc.collect()

In [ ]:
from postproc import runStatTests
alt = 'two-sided'
# TODO: make more verbose in verbose = 0
df_ttest_res = runStatTests(df_b2b_plot, cols0, alt, 
                            verbose=0)

In [ ]:
verbose = 0
# takes a long time to finish
from scipy.stats import ttest_rel, ttest_1samp
alt = 'two-sided'
def f(df):
    vsd = {}
    vsl = []
    resd = {'stable_greater_random':np.nan}
    
    # choosing subject present for both env
    subjs = []
    for env_cur in ['stable', 'random']:        
        ss = df.query('env == @env_cur')['subject'].unique()
        subjs += [ss]
    subj_both = list(set(subjs[0]) & set(subjs[1]))
    
    for env_cur in ['stable', 'random']:        
        dftmp = df.query('env == @env_cur')            
        if len(dftmp) == 0:
            #print(f'zero len for {ct}: {varname}, skipping')
            continue
        assert len(dftmp) <= 20, len(dftmp)
        vsd[env_cur] = dftmp['vals_to_plot'].values
        
        dftmp = dftmp.query('subject in @subj_both')        
        vsl += [dftmp['vals_to_plot'].values]
    if len(vsd) == 0:
        return None
    for env_cur in ['stable', 'random']:
        if env_cur not in vsd:
            print(f'No {env_cur} found')
            print(df[cols0].iloc[0])
            resd = pd.DataFrame([resd])
            return resd
                
        tr = ttest_1samp( vsd[env_cur], 0, alternative=alt )
        s = ' '
        if tr.pvalue < 0.05:
            s = '*'
        #if (tr.pvalue < 0.05) or (not signif_only):
        #    print(s + '{} control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format(tmin, ct, varname, env_cur, tr.pvalue) )
        resd[env_cur] = tr.pvalue
        resd[env_cur + '_len'] = len(vsd[env_cur])
    if len(vsl[0]) != len(vsl[1]):
        print('Not equal!', (len(vsl[0]), len(vsl[1])) )
        print(len(subj_both))
        print(df[cols0].iloc[0])
        resd = pd.DataFrame([resd])
        return resd
    
    tr = ttest_rel(vsl[0],vsl[1], alternative=alt )
    
    s = ' '
    if tr.pvalue < 0.05:
        s = '*'
        #tmins_good += [tmin]
    #if (tr.pvalue < 0.05) or (not signif_only):
    #    print(s + '{} control={:8}, var={:14}, stable>random pvalue={:.4f}'.format(tmin, ct, varname, tr.pvalue) )
    #print('')
    if alt == 'greater':
        resd['stable_greater_random'] = tr.pvalue
    else:
        resd['stable_neq_random'] = tr.pvalue
    #resd['tmin'] = tmin
    #resd['varname'] = varname
    #resd['control_type'] = ct
    #resd['custom_suffix'] = suff
    #resd['varseti'] = varseti
    resd = pd.DataFrame([resd])
    #rows += [resd]
    return resd
df_ttest_res = df_b2b_plot.groupby(cols0).apply(f)
  

In [ ]:
df_ttest_res_sig = df_ttest_res.query('stable_neq_random <= 0.05')
df_ttest_res_sig2 = df_ttest_res.query('stable_neq_random <= 0.05 and stable <= 0.05')

In [ ]:
df_ttest_res_basic = df_ttest_res.query('varseti == 0')
df_ttest_res_ES = df_ttest_res.query('varseti == 1')

display( df_ttest_res_ES['stable_len'].describe(),
        df_ttest_res_basic['stable_len'].describe() )

#df_ttest_res_sig = df_ttest_res.query('stable_greater_random <= 0.05')
print(len(df_ttest_res_sig))

In [ ]:
#df_ttest_res.index._names, df_ttest_res.columns

In [ ]:
# Do T-tests (old)
#suff = 'trim0_dhittw1_s_scX0Y0_drall_dtall'
#suff = 'CNhome_trim0_dhittw1_s_scX1Y1_drall_dtall'
signif_only = 1
rows = []
for varseti in [0,1]:
    qs = ('varseti == @varseti and custom_suffix == @suff')
    for suff in suffixes_present:
        print('----------- ' ,suff)    
        for ct in control_type_present: #, 'feedback']:
            for varname in ['err_sens', 'correction', 
                    'prev_error', 'error', 'prev_belief', 'prev_movement', 'prev_feedback']:
                tmins_good = []
                for tmin in tmins_to_plot:
                    vsd = {}
                    vsl = []
                    resd = {}
                    for env_cur in ['stable', 'random']:
                        qs2 = qs + ' and env == @env_cur and control_type == @ct and varname == @varname'
                        qs2 += ' and tmin == @tmin'
                        dftmp = df_b2b_plot.query(qs2)            
                        if len(dftmp) == 0:
                            #print(f'zero len for {ct}: {varname}, skipping')
                            continue
                        assert len(dftmp) <= 20, len(dftmp)
                        vsd[env_cur] = dftmp['vals_to_plot'].values
                        vsl += [dftmp['vals_to_plot'].values]
                    if len(vsd) == 0:
                        continue
                    for env_cur in ['stable', 'random']:
                        tr = ttest_1samp( vsd[env_cur], 0, alternative='greater' )
                        s = ' '
                        if tr.pvalue < 0.05:
                            s = '*'
                        if (tr.pvalue < 0.05) or (not signif_only):
                            print(s + '{} control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format(tmin, ct, varname, env_cur, tr.pvalue) )
                        resd[env_cur] = tr.pvalue
                        resd[env_cur + '_len'] = len(vsd[env_cur])
                    tr = ttest_rel(vsl[0],vsl[1], alternative='greater')
                    s = ' '
                    if tr.pvalue < 0.05:
                        s = '*'
                        tmins_good += [tmin]
                    if (tr.pvalue < 0.05) or (not signif_only):
                        print(s + '{} control={:8}, var={:14}, stable>random pvalue={:.4f}'.format(tmin, ct, varname, tr.pvalue) )
                    print('')
                    resd['stable_greater_random'] = tr.pvalue
                    resd['tmin'] = tmin
                    resd['varname'] = varname
                    resd['control_type'] = ct
                    resd['custom_suffix'] = suff
                    resd['varseti'] = varseti
                    rows += [resd]
            
            

df_ttest_res = pd.DataFrame(rows)


In [ ]:
cols0

In [ ]:
# just to see numbers, not used later
df_ttest_res_ES_sig = df_ttest_res_ES.query('stable_neq_random <= 0.05').reset_index()
display( df_ttest_res_ES_sig.groupby(cols0[:-1]).size().max() )
df_ttest_res_basic_sig = df_ttest_res_basic.query('stable_neq_random < 0.05').reset_index()
#df_ttest_res_basic_sig.groupby(['time_locked','custom_suffix','control_type','varname']).size()
display( df_ttest_res_basic_sig.groupby(cols0[:-1]).size().max() )

In [ ]:
#qs = 'stable_greater_random <= 0.05 and stable <= 0.05'
qs = 'stable_neq_random <= 0.05 and stable <= 0.05'
df_ttest_res_ES_sig2 = df_ttest_res_ES.query(qs)
df_ttest_res_basic_sig2 = df_ttest_res_basic.query(qs)

display( df_ttest_res_ES_sig2.groupby(cols0[:-1]).size() )
display( df_ttest_res_basic_sig2.groupby(cols0[:-1]).size() )

### dbg

In [ ]:
#set(tmins_good) ^ set(tmins_to_plot), len((tmins_to_plot)), len(tmins_good)

In [ ]:
#
#dfconddif_pv.sort_values?
#dfconddif_pv
#dfconddif_pv.loc[(0, varname)]#.query('signif == True')#['tmin']
#dfconddif_pv
#dfconddif0_pv

In [ ]:
df_ttest_res_sig.index._names

In [ ]:
cols0[:-1]

In [ ]:
# indcols = df_ttest_res_sig.index._names[:-1]
# for df_ in dfconddif0_pv, dfconddif_pv, df_ttest_res_sig:
#     df_.reset_index(drop=False, inplace=True)
#     df_.set_index(indcols, inplace=True)

In [ ]:
#tpl = (tl, ct, 'stable',slice(None), suff, varseti,'b2b', varname)

In [ ]:
#suffixes_present

In [ ]:
#df_ttest_res_sig.index._names

In [ ]:
#df_ttest_res_sig = df_ttest_res_sig.reset_index()

In [ ]:
#dec_types_found

In [ ]:
#dfsub = dfconddif_pv.loc[tpl].reset_index(drop=False)

In [ ]:
tpl

In [ ]:
df_ttest_res_sig.loc[tpl]

In [ ]:
gc.collect()

In [ ]:
dec_types_found

In [ ]:
df_b2b_plot.groupby(['subject','custom_suffix','dec_type']).size()

In [ ]:
df_b2b_plot.groupby('dec_type').size()

In [ ]:
dec_type = 'classic'

In [ ]:
suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall_broad_sh0.232'

In [ ]:
suff,tl,dec_type,varseti

In [ ]:
df_ = df_b2b_plot_.query(qs + ' and varseti == @varseti')
len(df_)

## plot

In [ ]:
del df_b2b_plot_

In [ ]:
gc.collect()

In [ ]:
qs_ = qs + ' and varseti == 0'
df_ = df_b2b_plot_.query(qs_)
df_g = df_[~df_['vals_to_plot'].isna()]
df_b = df_[df_['vals_to_plot'].isna()]
print(len(df_g),len(df_b))

In [ ]:
qs_ = qs + ' and varseti == -1'
df_ = df_b2b_plot_.query(qs_)
df_g = df_[~df_['vals_to_plot'].isna()]
df_b = df_[df_['vals_to_plot'].isna()]
print(len(df_g),len(df_b))

In [ ]:
sns.relplot(data=df_g, **pargs)

In [ ]:
assert 

In [ ]:
dur = 0.464
-0.5 - dur * 10, -0.5 + dur * 12

In [ ]:
df_b2b_plot.groupby('rt').size()

In [ ]:
list(df_b2b_plot_.varset.unique())

In [ ]:
# main plot
import re
df_b2b_plot_ = df_b2b_plot.query('rt == "Ridge"').copy()
df_b2b_plot_['tmin'] = df_b2b_plot_['tmin'].astype(float)

In [ ]:
df_b2b_plot.varseti.max()

In [ ]:
# time_locked = 'target'; dec_type='b2b'; varseti=-1
# df_b2b_plot_.query(qs_)['varname'].unique()

In [ ]:
cols0

In [ ]:
path

In [ ]:
#df_b2b_plot.to_picklekle('$')

In [ ]:
(df_b2b_plot.varname.unique())

In [ ]:
vns = ['prev_error','error','error_pred_Herz','error_pred_Tan',
       'error_pred_Died', 
       'err_sens','err_sens_Herz','err_sens_Tan']
vns = vns[1:]

In [ ]:
#df_b2b_plot.to_pickle(pjoin(path_data,'df_b2b_plot__for_Bernstein.pkl.zip'),compression='zip')

In [ ]:
list(suffixes_present)

In [ ]:
varn2pub1={'error':'Obeserved error', 
          'error_pred_Herz':'Herzfeld error prediction',
         'error_pred_Tan':'Tan error prediction',
         'error_pred_Died':'Diedrichsen error prediction'}
varn2pub2={'err_sens':'Empirical ES', 
          'err_sens_Herz':'Herzfeld ES',
         'err_sens_Tan':'Tan ES'}

In [ ]:
varn2pub = varn2pub1
vns_short = list(varn2pub.keys())
fign_suff = 'ERRslide'
vns_short

In [ ]:
varn2pub = varn2pub2
vns_short = list(varn2pub.keys())
vns_short
fign_suff = 'ESslide'

In [ ]:
trial_dur = 3.9145
mvt_dur = 0.7558
fb_dur = 0.25
ITI_dur = 1.5
home_dur = 0.5
wnd_SPoC_dur = 0.464
xvl = 0

In [ ]:
#dfconddif_pv['tmin'] = dfconddif_pv['tmin'].astype(float)
#dfconddif_pv= dfconddif_pv.sort_values('tmin')

dfconddif0_pv = dfconddif0_pv.reset_index().set_index(cols0[:-1])
dfconddif0r_pv = dfconddif0r_pv.reset_index().set_index(cols0[:-1])
dfconddif_pv = dfconddif_pv.reset_index().set_index(cols0[:-1])
df_ttest_res_sig= df_ttest_res_sig.reset_index().set_index(cols0[:-1])
#df_ttest_res_sig2 = df_ttest_res_sig2.reset_index().set_index(cols0[:-1])

import seaborn as sns; 
sns.set(font_scale=2.5)
sns.set_style( {'legend.frameon':True})
pargs = dict(kind='line',  col='control_type',  row='varname',
           x = 'tmin', y = 'vals_to_plot',   hue='env', 
             hue_order = ['stable','random'], errorbar='se',    
             palette=['orange', 'grey'], aspect=3.5, legend=False,
             facet_kws={'sharey': False, 'sharex': True} )

#for suff in  suffixes_present:
for suff in ['CNslideModelES_trim0_dhittw1_s_scX0Y0c1_drall_dtall_broad_sh0.250']:
    #for tl in time_lockeds_present:  
    for tl in ['feedback']:  
        for dec_type in dec_types_found:
            #qs = 'control_type == @ct and custom_suffix == @suff'
            qs = ('time_locked == @tl and custom_suffix == @suff '
                'and dec_type == @dec_type')
            suff_short = suff[14:].replace('_drall_dtall','').\
                replace('h0.250','').replace('_s','').replace('dhittw1cX0Y0c1','').replace('trim0','')

            #tick_skip = 2
            fignames = ['classic','basic_vars','ES']
            for varseti, fign in zip([-1, 0,1],fignames ):
#             fignames = ['classic_red']
#             for varseti, fign in zip([-1],fignames ):
                fign_full = f'{dec_type}_{fign_suff}_relplot_' +\
                    f'{suff_short}_{tl}_{fign}.pdf'
                qs_ = qs + ' and varseti == @varseti'
                if varseti == -1:
                    qs_ += ' and varname in @vns_short'
                    pargs['row_order'] = vns_short
                else:
                    pargs['row_order'] = None
                df_ = df_b2b_plot_.query(qs_)
                df_g = df_[~df_['vals_to_plot'].isna()]
                if not len(df_):
                    continue
                try:
                    #fg = sns.relplot(data=df_g, **pargs)
                    fg = sns.relplot(data=df_g, **pargs)
                except KeyError as e:
                    print('Err during relplot ',suff,tl,
                          dec_type,varseti,e)
                    plt.close()
                    continue
                axsf = fg.axes.flatten()
                for axi,ax in enumerate(axsf):
                    ax.axhline(y=0, c='r', ls=':', lw=3)
                    ax.axvline(x=xvl, c='r', ls=':', lw=3)
                    if tl == 'feedback':
                        ax.axvline(x=xvl - mvt_dur - home_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl - mvt_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl + fb_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl + fb_dur + ITI_dur, c='b', ls=':', lw=3)                    
                    elif tl == 'target':
                        ax.axvline(x=xvl - home_dur - ITI_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl - home_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl + mvt_dur, c='b', ls=':', lw=3)                    
                        ax.axvline(x=xvl + mvt_dur + fb_dur, c='b', ls=':', lw=3)  
                    ax.set_ylabel('Decoding accuracy')
                    ttl = ax.title.get_text()
                    mr = re.match('varname = (\w*) \| control_type = (\w*)', ttl)
                    if mr is None:
                        continue
                    mrg = mr.groups()
                    varname = mrg[0]
                    ct = mrg[1]
                    

                    if varseti == -1:
                        ax.set_title(varn2pub[varname])
                        if varname.find('err_sens') >= 0:                        
                            ax.set_ylim(-0.05,0.2)
                        else:
                            ax.set_ylim(-0.05,0.45)
                    else:
                        ax.set_ylim(-0.01,0.1)
                        
                    ymin,ymax = ax.get_ylim()
                    ymax *= 0.73

                    tpl = ('Ridge',suff,tl,ct,dec_type,varseti, varname)
                    # * markers are individ ttest > 0 and stable > random
                    try:
                        tmins_good = df_ttest_res_sig.loc[tpl]  
                        tmins_good = tmins_good.reset_index(drop=False)['tmin'].astype(float)
#                         ax.plot( tmins_good, 
#                                 len(tmins_good) * [ymax * 1.1], lw=0, 
#                                 marker='*', label='stable > random naive')
                    except KeyError as e:
                        tmins_good = []
                        
                    # x markers are individ ttest > 0 and stable > random
                    try:
                        tmins_stable_nonz = dfconddif0_pv.loc[tpl].query('signif == True').reset_index(drop=False)
                        tmins_stable_nonz = tmins_stable_nonz['tmin']
                        ax.plot( tmins_stable_nonz.astype(float), 
                            len(tmins_stable_nonz) * [ymax * 1.3], 
                            lw=0, marker='x', label='stable > 0')    
                        
                        tmins_random_nonz = dfconddif0r_pv.loc[tpl].query('signif == True').reset_index(drop=False)
                        tmins_random_nonz = tmins_random_nonz['tmin']
                        ax.plot( tmins_random_nonz.astype(float), 
                            len(tmins_random_nonz) * [ymax * 1.2], 
                            lw=0, marker='8', label='random > 0') 
                    except KeyError as e  :
                        tmins_stable_nonz = []

                    try:
                        dfsub = dfconddif_pv.loc[tpl].reset_index(drop=False)
                        dfsub['tmin'] = dfsub['tmin'].astype(float)
                        dfsub = dfsub.sort_values('tmin')
                        tmins_stable_g_random,sig = dfsub[['tmin','signif']].to_numpy().T                    
                        # plus markers are same as shading    
                        sig = sig.astype(bool)
                        tmins_stable_g_random = tmins_stable_g_random.astype(float)
                        #import matplotlib.patches as mpatches
                        from matplotlib.patches import Patch
                        fill= ax.fill_between(tmins_stable_g_random, 
                            ymax, where = sig, color='red', alpha=0.1)                        
#                         ax.plot( tmins_stable_g_random[sig], 
#                             np.sum(sig) * [ymax * 1.2], 
#                             lw=0, marker='+', label='stable != random')
                        ##poly = mcollections.PolyCollection(fill.get_paths(), 
                        #        facecolor='red', alpha=0.1,
                        #        label='stable != random')

                    except KeyError as e:
                        tmins_stable_g_random,sig = [],[]
                    print(varname, len(tmins_good), len(tmins_stable_g_random), len(tmins_stable_nonz))

                    #xticks = ax.get_xticks()[::tick_skip]
                    #ax.set_xticks(xticks)
                    ax.grid(True)
                    if axi == len(axsf) - 1:
                        ax.legend(loc='lower right',markerscale=2.5,
                                  framealpha=0.55, fontsize='medium')   
                        ax.set_xlabel('Window start [s]')
                    if axi == len(axsf) - 2:
                        llw = 10
                        from matplotlib.lines import Line2D
                        from matplotlib.text import Text
#                         text = Text(0.5, 0.5, 'eeddddddddddde', 
#                                     visible=True, label='environment')
#                         ax.add_artist(text)
                        l1 = Line2D([0], [0], color='orange', label='stable', lw=llw)
                        l2 = Line2D([0], [0], color='grey', label='random', lw=llw)
                        poly = Patch(facecolor='red', alpha=0.1,
                                label='stable !=\n random')
                        ax.legend(handles=[ l1, l2, poly], facecolor='white',
                                  #labels=['1','2','3'],
                            loc='lower right', framealpha=0.75, fontsize='medium') 
                    if axi == len(axsf) - 3:
                        llw = 6
                        from matplotlib.lines import Line2D
                        from matplotlib.text import Text
#                         text = Text(0.5, 0.5, 'eeddddddddddde', 
#                                     visible=True, label='environment')
#                         ax.add_artist(text)
                        l1 = Line2D([0], [0], color='r', label=tl, lw=llw, ls=':')
                        l2 = Line2D([0], [0], color='b', label='task stages', lw=llw, ls=':')
                        ax.legend(handles=[ l1, l2], facecolor='white',
                                  #labels=['1','2','3'],
                            loc='lower right', framealpha=0.75, fontsize='small')
                try:
                    #sns.move_legend(fg, loc=(0.75,0.),
                    #               facecolor='white', edgecolor='k') 
                    #sns.move_legend(fg, loc=(0.85,0.22))
                    #sns.move_legend(fg, loc='lower right')
                    #    edgecolor='black', markerscale=5., framealpha=1.)
                    #plt.suptitle(f'{dec_type} time locked = {tl}', x=0.2)
                    plt.tight_layout()
                    plt.savefig( pjoin(path_fig, fign_full), 
                                metadata = {'suff':suff, 'tl':tl})
                    plt.savefig( pjoin(path_fig, fign_full.replace('.pdf','.png')), 
                                metadata = {'suff':suff, 'tl':tl})
                    plt.close()
                except ValueError as e:
                    print('!!! EXC',e)
                print(f'{dec_type} varseti={varseti} fig done')

            break
        break
    break

plt.clf()
plt.close()

In [ ]:


# Add the legend
ax.legend(handles=[l1, l2], loc='upper left', frameon=True, facecolor='blue')

In [ ]:
gc.collect()

In [ ]:
dur = 0.464 
ds = np.arange( -dur*.5 * 64, dur*.5 * 65 , dur/2 ) - 0.5 + dur


In [ ]:
ds[0], ds[-1]

In [ ]:
ds[60:-60] 

In [ ]:
print(ttl)
mrg = re.match('varname = (\w*) | control_type = (\w*)', ttl).groups()
mrg

In [ ]:
0.464 / 4

## Many boxplots

In [ ]:
sns.set(font_scale=1.5)

import seaborn as sns
#suff = 'CNslide_trim0_dhittw1_s_scX0Y0_drall_dtall'
suff = 'CNslide_trim0_dhittw0_s_scX1Y1_drall_dtall'
qs = 'tmin.isin(@tmins_to_plot) and control_type == "movement" and custom_suffix == @suff'
pargs = dict(kind='boxen',  col='custom_suffix',  row='varname',
           x = 'tmin', y = 'vals_to_plot',   hue='env', 
             palette=['orange', 'grey'], aspect=9,
            sharey = None, sharex = None)

tick_skip = 3

fg = sns.catplot(data=df_b2b_plot.query(qs + ' and varseti == 0'), **pargs)
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':', lw=3)
    ax.axvline(x=indhome, c='r', ls=':', lw=3)
    ax.grid()
    varname = re.match('varname = (\w*) |.*', ax.title.get_text() ).groups()[0]
    tmins_good = df_ttest_res_basic_sig.query('varname == @varname')['tmin'].values    
    tmins_good2 = dfconddif_pv.loc[(suff, 0, varname)].query('signif == True')['tmin']
    
    ymin,ymax = ax.get_ylim()
    ax.plot( tmins_good, len(tmins_good) * [ymax * 1.1], lw=0, marker='*')
    ax.plot( tmins_good2, len(tmins_good2) * [ymax * 1.3], lw=0, marker='+')
    print(varname, len(tmins_good2))
    
    xticks = ax.get_xticks()[::tick_skip]
    ax.set_xticks(xticks)
sns.move_legend(fg, loc='upper right')
plt.tight_layout()
plt.savefig( pjoin(path_fig, 'b2b_slide_basic_vars.pdf'))
print('1st fig done')

fg = sns.catplot(data=df_b2b_plot.query(qs + ' and varseti == 1'), **pargs)
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':', lw=3)
    ax.axvline(x=indhome, c='r', ls=':', lw=3)
    ax.grid()
    varname = re.match('varname = (\w*) |.*', ax.title.get_text() ).groups()[0]
    tmins_good= df_ttest_res_ES_sig.query('varname == @varname')['tmin'].values
    tmins_good2 = dfconddif_pv.loc[(suff, 1, varname)].query('signif == True')['tmin']
    ymin,ymax = ax.get_ylim()
    ax.plot( tmins_good, len(tmins_good) * [ymax * 1.1], lw=0, marker='*')
    ax.plot( tmins_good2, len(tmins_good2) * [ymax * 1.3], lw=0, marker='+')
    print(varname, len(tmins_good2))
    
    xticks = ax.get_xticks()[::tick_skip]
    ax.set_xticks(xticks)


sns.move_legend(fg, loc='upper right')
plt.tight_layout()
plt.savefig( pjoin(path_fig, 'b2b_slide_ES.pdf'))

In [ ]:
ax.get_xticks()

## Plot home from slide

In [ ]:
suff = 'CNslideModelES_trim0_dhittw1_s_scX0Y0c1_drall_dtall_broad_sh0.250'
#tmin0 = '-0.26'
tmin0 = '0.11'
df_ = df_b2b_plot.query(
        'varseti == 0 and custom_suffix == @suff and tmin == @tmin0')
len(df_)

In [ ]:
sorted(df_b2b_plot_.tmin.unique())

In [ ]:
df_b2b_plot.varseti.unique()

In [ ]:
len(df_b2b_plot.query('varseti == 0') )

In [ ]:
#list(ts__)

In [ ]:
ts__ = df_b2b_plot['tmin'].unique()
ts__

In [ ]:
#tmins_to_plot

In [ ]:
len(df_)

In [ ]:
lbd = lambda x: x[14:].replace('_drall_dtall','').\
    replace('_sh0.232','').replace('_s','').replace('_trim0_','').\
    replace('h0.250','').replace('dhittw1cX0Y0c1_','')
df_b2b_plot['suff_short'] = df_b2b_plot['custom_suffix'].apply(lbd,1)

In [ ]:
suff_short

In [ ]:
list(df_b2b_plot['suff_short'].unique())

In [ ]:
df_.groupby('suff_short').size()
df_.query('suff_short == "dhittw1cX0Y0c1_gammah0.250" and varname.isin(@vns)')

In [ ]:
roword = [ 'dhittw1cX0Y0c1_broadh0.250',
  'dhittw1cX0Y0c1_betah0.250',
  'dhittw1cX0Y0c1_gammah0.250']
roword = [ 'broad',
  'beta',
  'gamma']

In [ ]:
varn2pub = varn2pub2.copy()
varn2pub.update(varn2pub1)
varn2pub

In [ ]:
formatter = lambda x: varn2pub[x]

In [ ]:
gc.collect()

In [ ]:
vns_

In [ ]:
tmins0

In [ ]:
sns.set(font_scale=1.8)
#tmin0 = '-0.64'  # -0.50
tmin0 = '-0.26'
#tmin0 = '0.11'

print('tmin0 = ', tmin0)
import seaborn as sns
pargs = dict(kind='box',  row='suff_short',  col='varseti',
               y = 'varname', x = 'vals_to_plot',
             hue='env', palette=['orange', 'grey'],
            hue_order= ['stable','random'], row_order=roword,
            order = vns)

#for suff in df_b2b_plot['custom_suffix'].unique():
suff_short = suff[14:].replace('_drall_dtall','').\
    replace('h0.250','').replace('_s','') 

for tl in ["target","feedback"]:
    for vns_,aspect in [(vns[:4], 1.6), (vns[4:], 1.6)]:
        df_ = df_b2b_plot.query('tmin == @tmin0 and varname.isin(@vns_)'
                                ' and time_locked == @tl and dec_type == "classic"')
        if len(df_) != 0:    
            pargs['order'] = vns_
            fg = sns.catplot(data=df_, **pargs, aspect=aspect)
            for ax in fg.axes.flatten():

                ttl = ax.get_title()
                mr = re.match('suff_short = (\w*) \| varseti = (\w*)', ttl)
                if mr is None:
                    continue
                mrg = mr.groups()
                suff = mrg[0]
                vi = mrg[1]

                ax.set_title(f'Frequency band = {suff}')

                ax.axvline(x=0, c='r', ls=':', lw=4)
                ax.grid(True)
                xlabs = ax.get_yticklabels()
                xlabs2 = []
                for xl in xlabs:
                    tt = xl.get_text()
                    xlabs2.append( varn2pub.get(tt,tt) )
                ax.set_yticklabels(xlabs2)

                ax.set_ylabel('')
                #ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

            fg.set_xlabels(f'Decoding accuracy at t={tmin0}')

            sns.move_legend(fg, loc=(0,-0.005) )#'lower left')
            #plt
            #plt.suptitle(suff_short)
            plt.tight_layout()    
            #plt.grid(True)

            #break

            fnfig = pjoin(path_fig, 
                f'b2b_home_basic_vars_{tmin0}_{suff_short}_{tl}_{",".join(vns_)}.pdf')
            plt.savefig( fnfig)
            plt.savefig( fnfig.replace('.pdf','.png'))

        #     df_ = df_b2b_plot.query('varseti == 1 and custom_suffix == @suff and tmin == "-0.50"')
        #     fg = sns.catplot(data=df_, **pargs)
        #     for ax in fg.axes.flatten():
        #         ax.axhline(y=0, c='r', ls=':')

        #     sns.move_legend(fg, loc='upper right')
        #     plt.suptitle(suff)
        #     plt.tight_layout()

        #     plt.savefig( pjoin(path_fig, f'b2b_home_ES_{tmin0}_{suff_short}.pdf'))
        print('finisehd ',suff_short)

In [ ]:
df_['tmin'] = 

In [ ]:
# herz is the only one who does okay in both stable and in random

In [ ]:
ttrssigs = []
ttrs_all = []
for tl in ['target', 'feedback']:
    #for suff_short in ['broad','beta','gamma']:
    for suff_short in ['broad']:
        for tmin0 in tmins0:
            for vn in vns:
                df_ = df_b2b_plot.query('suff_short == @suff_short and varname == @vn and'
                        ' tmin == @tmin0 and varseti == -1'
                        ' and time_locked == @tl and dec_type == "classic"')

                varn = 'vals_to_plot'
        #         st = df_.query('env == "stable"')#[varn].values
        #         rnd = df_.query('env == "random"')#[varn].values
                from behav_proc import comparePairs, compare0
                ttrssig, ttrs = comparePairs(df_, varn, 'env', paired = True)
                if ttrs is not None:
                    ttrs['varname'] = vn
                    ttrs['tmin'] = tmin0
                    ttrs['env'] = 'stable vs random'
                    ttrs['type'] = 'comp_env'
                    ttrs['suff_short'] = suff_short
                    ttrs['tl'] = tl
                ttrs_all += [ttrs]

                for env in ['random','stable']:
                    ttrs0 = compare0(df_.query('env == @env'),varn)
                    ttrs0['varname'] = vn
                    ttrs0['tmin'] = tmin0
                    ttrs0['env'] = env            
                    ttrs0['type'] = 'check_nz'
                    ttrs0['pooled'] = False
                    ttrs0['suff_short'] = suff_short
                    ttrs0['tl'] = tl
                    #print(len(ttrs0))
                    ttrs_all += [ttrs0]

            for env in ['random','stable']:
                df_ = df_b2b_plot.query('suff_short == "broad" and varname.isin(@vns) and'
                    ' tmin == @tmin0 and varseti == -1'
                    ' and time_locked == "feedback" and dec_type == "classic"'
                    ' and env == @env')
                ttrssig, ttrs = comparePairs(df_, varn, 'varname', paired = True)
                ttrs['varname'] = None
                ttrs['tmin'] = tmin0
                ttrs['env'] = env
                ttrs['type'] = 'comp_var'
                ttrs['suff_short'] = suff_short
                ttrs['tl'] = tl
                ttrs_all += [ttrs]          
            
    #ttrssigs = pd.concat(ttrssigs, ignore_index=1).query('pooled == False')

ttrs_all = pd.concat(ttrs_all, ignore_index=1).query('pooled == False')

In [ ]:
ttrs_allg = ttrs_all.query('pval < 0.05 and alternative != "two-sided"')

In [ ]:
ttrs_allg.sort_values(['type','varname']).\
    query('tmin == "0.11" and type != "comp_var" and varname.isin(["error_pred_Died","error_pred_Herz"])')\
    [['type','tmin','tl','env','varname','ttstr','pval','T']]

In [ ]:
varpairs = [('error','prev_error'), 
            ('err_sens','err_sens_Tan'), ('err_sens','err_sens_Herz'), 
            ('err_sens_Tan','err_sens_Herz'), 
('error','error_pred_Tan'), ('error','error_pred_Herz'), 
            ('error','error_pred_Died'), ('error_pred_Herz','error_pred_Died'),
           ('error_pred_Tan','error_pred_Died')]

for tmin in tmins0:
    print('\n      ',tmin)
    for v1,v2 in varpairs:    
        r = ttrs_allg.query('tmin == @tmin and type == "comp_var" '
                        'and val1 == @v1 and val2 == @v2')
        print(v1,v2, len(r))
        if len(r):
            display(r[['type','tmin','tl','env','varname','ttstr','pval','T']])

In [ ]:
ttrs_all['val2'].unique()

In [ ]:
ttrs_all.query('val1 == "error_pred_Died" and val2 == "error_pred_Herz" and env == "random"')\
    [['type','tmin','tl','env','varname','ttstr','pval','T']]

In [ ]:
df_.groupby(['subject','env']).size().max()

In [ ]:
sns.set(font_scale=1.1)
#tmin0 = '-0.64'  # -0.50
tmins0 = ['-0.26','0.11']
#tmin0 = '0.11'
print('tmin0 = ', tmin0)
import seaborn as sns
pargs = dict(kind='box',  row='suff_short',  col='tmin',
               y = 'varname', x = 'vals_to_plot',
             hue='env', palette=['orange', 'grey'],
            hue_order= ['stable','random'], row_order=roword,
             col_order = tmins0,
            order = vns)

#for suff in df_b2b_plot['custom_suffix'].unique():
suff_short = suff[14:].replace('_drall_dtall','').\
    replace('h0.250','').replace('_s','') 

df_ = df_b2b_plot.query('tmin.isin(@tmins0) and varname.isin(@vns)'
                        ' and varseti == -1 and time_locked == "feedback"'
                         ' and dec_type == "classic"')
if len(df_) != 0:    
    fg = sns.catplot(data=df_, **pargs)
    for ax in fg.axes.flatten():
        ax.axvline(x=0, c='r', ls=':')
        ax.grid(True)

# Plot mini slide

In [ ]:
#for subject in subjects[:1]:
#%debug
import gc; gc.collect()
plot_diff_vs_es = 0
plot_diff = 0

regression_type = 'Ridge'
#control_type = 'movement'
#subject_inds = [0]
subject_inds = np.arange(len(subjects))
#subject_inds = list(range(12)) + list(range(13,20))
#subject_inds = [12]

# scale robust = 1
#time_start = parser.parse('14:40 of 31 may 2023'); 
#time_end = parser.parse('21:39 of 31 may 2023'); 

# scale robust = 0
#time_start = parser.parse('13:00 of 14 june 2023'); 
#time_start = parser.parse('11:36 of 20 june 2023'); 
time_start = parser.parse('17:00 of 27 june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 

custom_prefix = None

use_preload_df = 0
load_df        = 0
exit_after = 'collect'
plot_scores_per_subj = 0
ktes = ['err_sens','err_sens_Tan','err_sens_Herz',
        'error', 'error_pred_Tan','error_pred_Herz',
        'error_pred_Died', 
        'prev_error',
        'target_inds',
        'trials','trialwb', 'trial_wpert',
        'trialwtgt_wb','trialwtgt_wpertstage_wb',
        'RT','movement_duration']
#b2bdef = 'err_sens,prev_error,error'
b2bdef = ['err_sens,prev_error,error',
         'err_sens_Tan,prev_error,error',
         'err_sens_Herz,prev_error,error',]
#b2bdef = 'err_sens_Tan,err_sens_Herz,err_sens,prev_error,error'
control_types = ['movement', 'feedback']
varnames = ktes
#tmins = ['-5.00']
tmin_desired = '-0.50'
#tmin_desired = None
freq_names = ['broad','beta','gamma']
time_lockeds=['target', 'feedback']
#time_lockeds=['target']

#control_types = ['movement']
#freq_names = ['broad']
#time_lockeds = ['target']

import traceback
DEBUG = 1
require_all_subjects = 0

save_df = 0
#%debug

failed = []
dfs = []
for hpass in ['no_filter']: #,'0.1']:
    #for env in ['stable','random']:
    for env in [['stable','random']]:
        #for freq_name in list(freq_name2freq.keys() ):
        for freq_name in freq_names: #, 'beta']:
            for time_locked in time_lockeds:
                for control_type in control_types:
                    s=  ','.join( getAnalysisVarnames(time_locked, control_type)[0] )
                    varnames_b2b = [s ] + b2bdef

                    output_folder = f'corr_spoc_es_sliding2_{hpass}'        
                    script_name = pjoin(scripts_dir,
                            'correlation_error_sensitivity_spoc_decoding_sliding.py')
                    try:
                        ipy.magic('%run -i {script_name}')                  
                    except Exception as e:                    
                        print('EXC',e)
                        exc_info = sys.exc_info()
                        exc = traceback.TracebackException(*exc_info, capture_locals=True)
                        ei = exc_info[2]    
                        #display(ei.tb_frame)
                        psf = ei

                        lfprev = None; lf = None
                        lfs = []
                        di = 0
                        while psf.tb_frame is not None:
                            stackframe = psf.tb_frame
                            display(di, stackframe)

                            psf = psf.tb_next  
                            if 'conda' not in stackframe.f_code.co_filename:        
                                lfprev = lf
                                lf = stackframe.f_locals
                                lfs += [lf]
                                di += 1
                            if psf is None:
                                break    

                        raise e

                        print('Exc for ', hpass,freq_name)
                        failed += [ (hpass, freq_name)]
                        print('EXC',e)
                    #df.to_pickle( pjoin(path_fig, save_folder, 'df') )
                    dfs += [df]
                if not DEBUG:
                    del df
                import gc; gc.collect()
                plt.close()
            #break
        #break
    #break

dfc =pd.concat(dfs).reset_index()
print(len(dfc))
print(dfc['subject'].unique())
suffixes_present = dfc['custom_suffix'].unique()
print(suffixes_present)
dfc_types_present = dfc['dec_type'].unique(); 
print(dfc_types_present)

# Plot home

In [ ]:
# to test collection first exectue with small number of subject_inds
#%debug
import gc; gc.collect()
plot_diff_vs_es = 0
plot_diff = 0

regression_type = 'Ridge_noCV'
#control_type = 'movement'
subject_inds = [0]
#subject_inds = np.arange(len(subjects))

# scale robust = 0
#time_start = parser.parse('13:00 of 14 june 2023'); 
#time_start = parser.parse('11:36 of 20 june 2023'); 
time_start = parser.parse('19:30 of 27 june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 

custom_prefix = None

use_preload_df = 0
load_df        = 0
exit_after = 'collect'
plot_scores_per_subj = 0
ktes = ['err_sens','err_sens_Tan','err_sens_Herz',
        'error', 'error_pred_Tan','error_pred_Herz',
        'error_pred_Died', 
        'prev_error','next_error','prev_belief','belief',
        'prev_movement','movement',
        'target_inds',
        'trials','trialwb', 'trialwpert',
        'trialwtgt_wb','trialwtgt_wpertstage_wb',
        'RT','movement_duration']
#b2bdef = 'err_sens,prev_error,error'
b2bdef = ['err_sens,prev_error,error',
         'err_sens_Tan,prev_error,error',
         'err_sens_Herz,prev_error,error',]
#b2bdef = 'err_sens_Tan,err_sens_Herz,err_sens,prev_error,error'
control_types = ['movement', 'feedback']
varnames = ktes
#tmins = ['-5.00']
tmin_desired = '-0.50'
#tmin_desired = None
freq_names = ['broad','beta','gamma']
time_lockeds=['target', 'feedback']
#time_lockeds=['target']

#control_types = ['movement']
#freq_names = ['broad']
#time_lockeds = ['target']

import traceback
DEBUG = 1
require_all_subjects = 0

save_df = 0
#%debug

failed = []
dfs = []
#%debug
for hpass in ['no_filter']: #,'0.1']:
    #for env in ['stable','random']:
    for env in [['stable','random']]:
        #for freq_name in list(freq_name2freq.keys() ):
        for freq_name in freq_names: #, 'beta']:
            for time_locked in time_lockeds:
                for control_type in control_types:
                    s=  ','.join( getAnalysisVarnames(time_locked, control_type)[0] )
                    varnames_b2b = [s ] + b2bdef

                    output_folder = f'corr_spoc_es_sliding2_{hpass}'        
                    script_name = pjoin(scripts_dir,
                            'correlation_error_sensitivity_spoc_decoding_sliding.py')
                    try:
                        ipy.magic('%run -i {script_name}')                  
                    except Exception as e:                    
                        print('EXC',e)
                        exc_info = sys.exc_info()
                        exc = traceback.TracebackException(*exc_info, capture_locals=True)
                        ei = exc_info[2]    
                        #display(ei.tb_frame)
                        psf = ei

                        lfprev = None; lf = None
                        lfs = []
                        di = 0
                        while psf.tb_frame is not None:
                            stackframe = psf.tb_frame
                            display(di, stackframe)

                            psf = psf.tb_next  
                            if ('conda' not in stackframe.f_code.co_filename) and\
                                 'mamba' not in stackframe.f_code.co_filename:        
                                lfprev = lf
                                lf = stackframe.f_locals
                                lfs += [lf]
                                di += 1
                            if psf is None:
                                break    

                        raise e

                        print('Exc for ', hpass,freq_name)
                        failed += [ (hpass, freq_name)]
                        print('EXC',e)
                        df = None
                    #df.to_pickle( pjoin(path_fig, save_folder, 'df') )
                    dfs += [df]
                if not DEBUG:
                    del df
                import gc; gc.collect()
                plt.close()
            #break
        #break
    #break

dfc =pd.concat(dfs).reset_index()
print(len(dfc))
print(dfc['subject'].unique())
suffixes_present = dfc['custom_suffix'].unique()
print(suffixes_present)
dfc_types_present = dfc['dec_type'].unique(); 
print(dfc_types_present)

In [ ]:
dfc =pd.concat(dfs).reset_index()
print(len(dfc))
print(dfc['subject'].unique())
suffixes_present = dfc['custom_suffix'].unique()
print(suffixes_present)
dfc_types_present = dfc['dec_type'].unique(); 
print(dfc_types_present)

In [ ]:
#%debug
with warnings.catch_warnings(record=True) as w:
    warnings.filterwarnings('error')

    ipy.magic('%run -i {script_name}') 

In [ ]:
lfs

In [ ]:
lfs[-1].keys()

In [ ]:
from postproc import collectResultsPrefilled,collectResults0
hpass = 'no_filter'
time_start = parser.parse('19:30 of 27 june 2023'); 
time_end = parser.parse('15:58 of 31 may 2027'); 
time_lockeds = ['target', 'feedback']
freq_names = ['broad']
#freq_names = ['broad', 'beta', 'gamma']
control_types = [ 'movement']

output_folder = f'corr_spoc_es_sliding2_{hpass}'        
df1 = collectResults0(subjects,output_folder)

qs = 'time_locked.isin(@time_lockeds) and control_type.isin(@control_types)'
qs += ' and freq_name.isin(@freq_names)'
qs += ' and mtime >= @time_start'
qs += ' and mtime <= @time_end'
if tmin_desired is not None:
    if isinstance(tmin_desired,list):
        qs += ' and tmin.isin(@tmin_desired)'
    else:
        qs += ' and tmin == @tmin_desired'
df2 = df1.query(qs).copy()
print('All files {},  after filtering {}'.format(len(df1),len(df2)) )

In [ ]:
# slow
#%debug
dfc = collectResultsPrefilled(df2)
dfc_allcollected = dfc

catcols = ['subject', 'fn', 'fn_full', 'SLURM_job_id', 'env',
       'trial_group_col_calc', 'rt', 'time_locked', 'control_type',
       'custom_suffix', 'freq_name', 'tmin', 'tmax', 'dec_type']
for col in catcols:
    dfc[col] = dfc[col].astype('category')

In [ ]:
del df1,df2

In [ ]:
#dfc_home = dfc

In [ ]:
dfc.describe(include='category').T

In [ ]:
f = np.load( dfc['fn_full'].iloc[0], allow_pickle=1 )
d = f['decoding_per_var'][()]

In [ ]:
print( f.keys() )

In [ ]:
d.keys()

In [ ]:
list(d.keys())

In [ ]:
list(dfc.columns)

In [ ]:
suffixes_present

In [ ]:
print (set( df['subject'] ) ^ set(np.array(subjects)[subject_inds]))

In [ ]:
dfc['mtime'].min(), dfc['mtime'].max()

In [ ]:
# dubplicates check
cols_unique = ['subject','env','time_locked','control_type', 
    'trial_group_col_calc','freq_name', 'tmin', 'tmax', 'dec_type',
              'custom_suffix']
if dfc.duplicated(cols_unique).any():
    display( dfc[ dfc.duplicated(cols_unique) ] )
else:
    print('No duplicates!')

print( len(dfc), dfc['subject'].nunique(), dfc['control_type'].nunique(), subject_inds )

In [ ]:
grp = dfc.groupby(cols_unique)
print(grp.size().max())

In [ ]:
#dfc = dfc.drop_duplicates(cols_unique)

In [ ]:
list(dfc.columns)

In [ ]:
g = list(grp.groups.keys())[0]
inds = grp.groups[g]
dfc.loc[inds, ['SLURM_job_id','err_sens_Xshape']]

In [ ]:
varsets = dfc.query('dec_type == "classic"')['varset'].unique()
print(list(varsets))
varnames0 = list(set(';'.join(varsets).split(';') ) )#.split(',')
print('varnames0',varnames0)

In [ ]:
from postproc import  extractB2Bcols
#%debug
#varnames0 = ['prev_movement','prev_belief','movement','belief','feedback',
#             'error','prev_error','err_sens','next_error','correction' ]
#varnames0 = ktes
dfc_, df_b2b_plot0 = extractB2Bcols(dfc, b2bdef,
                        toclean=varnames0)

suffixes_present = df_b2b_plot0['custom_suffix'].unique()
print(len(dfc_))

In [ ]:
[col for col in dfc.columns if col.find('score') >= 0]

In [ ]:
from postproc import extractClassicCols
#,'movement','feedback','next_error','prev_belief','belief',
#'prev_movement',
# varnames0 = ['error','error_pred_Tan','error_pred_Herz','error_pred_Died',
#              'error_pred_Died','prev_error',
#              'err_sens','err_sens_Tan','err_sens_Herz',
#              'trials','target_inds' ]
#varnames = ktes
df_class_plot = extractClassicCols(dfc, defvarcolns=varnames0,
                                  error_handling = 'ignore')

#df_b2b_plot = df_class_plot


In [ ]:
df_b2b_plot = pd.concat([df_class_plot,df_b2b_plot0] )#.reset_index()

In [ ]:
catcols = ['control_type', 'env', 'varname', 
           'time_locked', 'custom_suffix']
for col in catcols:
    df_b2b_plot0[col] = df_b2b_plot0[col] .astype('category')

In [ ]:
lbd = lambda x: x[14:].replace('_drall_dtall','').\
    replace('_sh0.232','').replace('_s','').replace('trim0_','')
df_b2b_plot['suff_short'] = df_b2b_plot['custom_suffix'].apply(lbd,1)

In [ ]:
varnameord = ['target_inds',
 'prev_error',
 'error',              
 'next_error', 
 'prev_movement',
 'movement',
 'prev_belief',
 'belief',
 'err_sens']
              
#  ['err_sens_Tan',
#  'err_sens_Herz',
#  'error_pred_Tan',
#  'error_pred_Herz',
#  'error_pred_Died']

In [ ]:
#list(df_b2b_plot['varname'].unique())

In [ ]:
#list(df_b2b_plot['suff_short'].unique())
# roword = ['dhittw1cX0Y0c0_broad', 
#           'dhittw1cX0Y0c1_broad',
#  'dhittw1cX0Y1c0_broad',
#  'dhittw1cX0Y1c1_broad',
#  'dhittw1cX0Y2c0_broad',
#  'dhittw1cX0Y2c1_broad']

roword = ['dhittw1cX0Y0c0_broad', 
          'dhittw1cX0Y0c1_broad',
          'dhittw1cX0Y2c0_broad',
          'dhittw1cX0Y2c1_broad',
          'dhittw1cX0Y1c0_broad',
          'dhittw1cX0Y1c1_broad' ]

In [ ]:
df_b2b_plot['suff_short']

In [ ]:
cols_unique

In [ ]:
dfc.duplicated(cols_unique).any()

In [ ]:
dfc['rt']

In [ ]:
df_.groupby(['varname','suff_short','time_locked','env',
            'control_type','subject','dec_type','varseti'])\
    .size().max()

In [ ]:
df_b2b_plot.query('varseti == 0')['varset'].unique()

In [ ]:
sns.set(font_scale=1.1)
tmin0 = '-0.50'  #'-0.64' 
import seaborn as sns
pargs = dict(kind='box',  row='suff_short',  col='varseti',
               y = 'varname', x = 'vals_to_plot',
             hue='env', palette=['orange', 'grey'],
            hue_order= ['stable','random'],aspect=1.2,
            order = varnameord, row_order = roword,
             sharex = 'col')#, row_order=roword)

for tl in time_lockeds:
    df_ = df_b2b_plot.query('tmin == @tmin0 and varname != "trials"'
                           ' and time_locked == @tl')
    assert not df_.duplicated(cols_unique + ['varname','varseti']).any()
    if len(df_) != 0:    
        fg = sns.catplot(data=df_, **pargs)
        for ax in fg.axes.flatten():
            ax.axvline(x=0, c='r', ls=':')
            ax.grid(True)
            #ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

        sns.move_legend(fg, loc='upper right')
        #plt
        #plt.suptitle(suff_short)
        plt.tight_layout()    
        #plt.grid(True)

        #break

        plt.savefig( pjoin(path_fig, f'classic_home_basic_vars_{tmin0}_{tl}.pdf'))


    #     df_ = df_b2b_plot.query('varseti == 1 and custom_suffix == @suff and tmin == "-0.50"')
    #     fg = sns.catplot(data=df_, **pargs)
    #     for ax in fg.axes.flatten():
    #         ax.axhline(y=0, c='r', ls=':')

    #     sns.move_legend(fg, loc='upper right')
    #     plt.suptitle(suff)
    #     plt.tight_layout()

    #     plt.savefig( pjoin(path_fig, f'b2b_home_ES_{tmin0}_{suff_short}.pdf'))
    print('finished ')

## for Bernstein abstract

In [ ]:
from postproc import extractClassicCols
#,'movement','feedback','next_error','prev_belief','belief',
#'prev_movement',
varnames0 = ['error','error_pred_Tan','error_pred_Herz','error_pred_Died',
             'error_pred_Died','prev_error',
             'err_sens','err_sens_Tan','err_sens_Herz',
             'trials' ]
varnames = ktes
df_class_plot = extractClassicCols(dfc, defvarcolns=varnames0,
                                  error_handling = 'ignore')

df_b2b_plot = df_class_plot

catcols = ['control_type', 'env', 'varname', 
           'time_locked', 'custom_suffix']
for col in catcols:
    df_b2b_plot[col] = df_b2b_plot[col] .astype('category')

In [ ]:
cols_ = ['subject',
 'time_locked',
 'control_type',
 'trial_group_col_calc',
 'freq_name',
 'tmin',
 'dec_type',
 'custom_suffix', 'varseti', 'varname', 'rt','trial_group_col_calc']

In [ ]:
list(df_b2b_plot['varname'].unique())

In [ ]:
df_b2b_plot = df_b2b_plot[cols_ + ['env', 'vals_to_plot'] ].drop_duplicates()

In [ ]:
df_b2b_plot = df_b2b_plot.reset_index()

In [ ]:
# dfsz = df_b2b_plot.groupby(cols_ + ['env']).size().to_frame()
# dfsz[dfsz[0] == 2]#.reset_index()

In [ ]:
#df_b2b_plot[df_b2b_plot.duplicated(cols_ + ['env'])]

In [ ]:
df_b2b_plot.query('env == "random"')

In [ ]:
catcols_noenv = list(set(catcols) - set([env]))

In [ ]:
catcols_noenv

In [ ]:
for col in cols_ :
    df_b2b_plot[col]= df_b2b_plot[col].astype('string')
df_b2b_plot = df_b2b_plot.reset_index()

In [ ]:
dfs = []
for env in ['stable','random']:
    df_ = df_b2b_plot.query('env == @env').copy()#.reset_index(drop=True)
    df_ = df_.drop(columns=['level_0','index','env','trial_group_col_calc'])
    
    for col in cols_  :
        if col not in df_:
            continue
        df_[col]= df_[col].astype('string')
        
    cols__ = list(set(cols_) - set(['trial_group_col_calc']) )
    dfs += [df_.set_index(cols__) ]

In [ ]:
df_dif = dfs[0]['vals_to_plot'] - dfs[1]['vals_to_plot']
df_dif = df_dif.reset_index()

In [ ]:
#df_b2b_plot.columns

In [ ]:
df_dif

In [ ]:
dfs[0].duplicated

In [ ]:
dfs['']

In [ ]:
#path_fit_R = pjoin( os.path.expandvars('$HOME'), 'ownCloud', 'Current' ,'_share_Romain' ); path_fit_R

In [ ]:
#df_b2b_plot['SLURM_job_id']

In [ ]:
dftmp['varname'].unique()

In [ ]:
from scipy.stats import ttest_rel

In [ ]:
# calc t-tests
#suff = 'trim0_dhittw1_s_scX0Y0_drall_dtall'
#suff = 'CNhome_trim0_dhittw1_s_scX1Y1_drall_dtall'
qs = ('varseti == 0 and '
 ' custom_suffix == @suff and tmin == "-0.50"')
signif_only = 1
for suff in suffixes_present:
    print('----------- ' ,suff)
    for ct in ['movement', 'feedback']:
        for varname in ['prev_movement', 'prev_error', 'error', 'prev_belief',
               'prev_feedback']:
            vsd = {}
            vsl = []
            for env_cur in ['stable', 'random']:
                qs2 = qs + ' and env == @env_cur and control_type == @ct and varname == @varname'
                dftmp = df_b2b_plot.query(qs2)            
                if len(dftmp) == 0:
                    #print(f'zero len for {ct}: {varname}, skipping')
                    continue
                assert len(dftmp) == 20, len(dftmp)
                vsd[env_cur] = dftmp['vals_to_plot'].values
                vsl += [dftmp['vals_to_plot'].values]
            if len(vsd) == 0:
                continue
            for env_cur in ['stable', 'random']:
                tr = ttest_1samp( vsd[env_cur], 0, alternative='greater' )
                s = ' '
                if tr.pvalue < 0.05:
                    s = '*'
                if (tr.pvalue < 0.05) or (not signif_only):
                    print(s + 'control={:8}, var={:14}, {}>0, pvalue={:.4f}'.format( ct, varname, env_cur, tr.pvalue) )
            tr = ttest_rel(vsl[0],vsl[1], alternative='greater')
            s = ' '
            if tr.pvalue < 0.05:
                s = '*'
            if (tr.pvalue < 0.05) or (not signif_only):
                print(s + 'control={:8}, var={:14}, stable>random pvalue={:.4f}'.format( ct, varname, tr.pvalue) )
            print('')
            

In [ ]:
path_fig

In [ ]:
df_b2b_plot['custom_suffix'].unique()

In [ ]:
#suffixes_desired = ['CNhome_trim0_dhittw1_s_scX0Y0c0_drall_dtall_broad']

In [ ]:
suffixes_desired = suffixes_present

In [ ]:
suffixes_present

In [ ]:
ttest_1samp?

In [ ]:
df_dif.columns

In [ ]:
vns = ["error_pred_Herz", "error_pred_Tan"]
dfs = []
for vn in vns:
    df_ = df_dif.query('varname in @vn').set_index(['control_type', 'subject', 'tmin', 'dec_type', 'varseti',
       'custom_suffix', 'rt', 'freq_name', 'time_locked'])
    dfs+=[df_]
    print( len(df_) )
    
dfdifdif_TH = dfs[0]['vals_to_plot'] - dfs[1]['vals_to_plot']
dfdifdif_TH = dfdifdif_TH.to_frame().reset_index()
for alt in ['two-sided','greater','less']:
    print(alt)
    def f(dftmp):
        vs = dftmp['vals_to_plot']
        vs = vs[~np.isnan(vs)]
        r = ttest_1samp(vs,0,alternative=alt)
        return r.pvalue
    dfr =  dfdifdif_TH.groupby(['control_type','time_locked','freq_name']).apply(f)
    display(dfr)

In [ ]:
assert not dfs[0].duplicated().any()
assert not dfs[1].duplicated().any()

In [ ]:
vns = ["err_sens_Herz", "err_sens_Tan"]
dfs = []
for vn in vns:
    df_ = df_dif.query('varname in @vn').set_index(['control_type', 'subject', 'tmin', 'dec_type', 'varseti',
       'custom_suffix', 'rt', 'freq_name', 'time_locked'])
    dfs+=[df_]
    print( len(df_) )
dfdifdif_TH = dfs[0]['vals_to_plot'] - dfs[1]['vals_to_plot']
dfdifdif_TH = dfdifdif_TH.to_frame().reset_index()
def f(dftmp):
    vs = dftmp['vals_to_plot']
    vs = vs[~np.isnan(vs)]
    r = ttest_1samp(vs,0,alternative='two-sided')
    return r.pvalue
dfdifdif_TH.groupby(['control_type','time_locked','freq_name']).apply(f)

In [ ]:
def f(dftmp):
    vs = dftmp['vals_to_plot'].values
    vs = vs[~np.isnan(vs)]
    r = ttest_1samp(vs,0,alternative='greater')
    return r.pvalue
dfsig = df_dif.groupby(['varname','control_type','time_locked','freq_name']).apply(f).to_frame()
#dfsig[dfsig]
dfsig[dfsig[0] < 0.05]

In [ ]:
vn2pub = dict(zip(['error_pred_Herz', 'error_pred_Tan', 'error_pred_Died'],
                  ['Herzfeld', 'Tan', 'Diedrichsen']))

In [ ]:
tls[0].get_text()

In [ ]:
# classic
varns = ['error_pred_Herz', 'error_pred_Tan', 'error_pred_Died']
sns.set(font_scale=2.1)
import seaborn as sns
#for suff in suffixes_desired:
pargs = dict(kind='box',               
           y = 'varname', x = 'vals_to_plot', row='freq_name',
              sharey='none',aspect=1.9)
fg = sns.catplot(data=df_dif.query(
    'tmin == "-0.50" and varname in @varns'
    ' and control_type == "feedback"'), 
                 **pargs)
for ax in fg.axes.flatten():
    #ax.axhline(y=0, c='r', ls=':')
    ax.set_xticklabels(ax.get_xticklabels())#, rotation=90)
    tls = ax.get_yticklabels()
    ax.set_yticklabels([ vn2pub[tl.get_text()] for tl in ax.get_yticklabels()] )#, rotation=90)
    ax.grid(True)
    ax.set_xlabel('decoding score stable - decoding score random')
    ax.set_ylabel('')
    ax.axvline(0,c='red',ls=':')

#sns.move_legend(fg, loc='upper right')
plt.suptitle(suff)
plt.tight_layout()    
plt.savefig( pjoin(path_fig, f'classic_diff_homeModelES.pdf'))

In [ ]:
#15:02

In [ ]:
# classic
sns.set(font_scale=1.1)
import seaborn as sns
for suff in suffixes_desired:
    
    pargs = dict(kind='box',  col='control_type',  
                 #row='varseti',
               x = 'varname', y = 'vals_to_plot',   hue='env',
                 palette=['orange', 'grey'] , sharey='none',
                 hue_order=['stable','random'],
                row='time_locked')
    fg = sns.catplot(data=df_b2b_plot.query(
        'custom_suffix == @suff and tmin == "-0.50"'), 
                     **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
        ax.grid(True)

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()    
    plt.savefig( pjoin(path_fig, f'classic_homeModelES_{suff}.pdf'))

In [ ]:
sns.set(font_scale=1.1)
import seaborn as sns
for suff in suffixes_desired:
    
    pargs = dict(kind='box',  col='control_type',  
                 #row='varseti',
               x = 'varname', y = 'vals_to_plot',   hue='env',
                 palette=['orange', 'grey'] , sharey='none',
                row='time_locked')
    fg = sns.catplot(data=df_b2b_plot.query(
        'varseti == 0 and custom_suffix == @suff and tmin == "-0.50"'), 
                     **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.grid(True)

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()    
    plt.savefig( pjoin(path_fig, f'b2b_home_basic_vars_{suff}.pdf'))

    fg = sns.catplot(data=df_b2b_plot.query('varseti == 1 and custom_suffix == @suff and tmin == "-0.50"'),
                     **pargs)
    for ax in fg.axes.flatten():
        ax.axhline(y=0, c='r', ls=':')
        ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
        ax.grid(True)

    sns.move_legend(fg, loc='upper right')
    plt.suptitle(suff)
    plt.tight_layout()

    plt.savefig( pjoin(path_fig, f'b2b_home_ES_{suff}.pdf'))

In [ ]:
#ct = "movement"
#env_cur = 'stable'
dfc_cl = dfc.query('dec_type == "classic" and tmin == "-0.50"')
dfc_cl = dfc_cl.copy()
print(len(dfc_cl))
fg = sns.catplot(data=dfc_cl, kind='boxen', x='control_type',
           y = 'prev_error_scores_mean_over_folds', 
                  hue='env')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':')
    
plt.suptitle('prev_error')
plt.savefig( pjoin(path_fig, 'classic_home_prev_error.pdf'))

### debug

In [ ]:
sub_df.reset_index()
print(len(sub_df))
sub_df

In [ ]:
list( sub_df['fn'] ) 

In [ ]:
times, fns, pars = list( zip(*srt) )
from datetime import datetime
days = [ datetime.fromtimestamp(t).day for t in times]
sum( np.array(days) < 20)

In [ ]:

#r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|movement)_(.*)_([a-z])_t=(.*)\.npz', fns[0])
#r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|target)_(.*)_([a-z]).*\.npz', fns[0])
r = re.match( r'(stable|random)_(Ridge|xgboost)_(feedback|target)_(.*)_(' +freqstr+ ')_t=(.*)\.npz', fns[0])
print(r )
grps = r.groups(); print(grps)
env,rt,tgt,analysis_name,freq_name,tminmax = grps

In [ ]:
len(days)

In [ ]:
fn_full

In [ ]:
shutil.

In [ ]:
# fns[0]
# fn_full = op.join(dir_full,fns[0])
# import shutil
# os.remove(fn_full)

In [ ]:
fn_full

In [ ]:
shift

In [ ]:
np.all( np.diff(times) > 0)

In [ ]:
d = datetime.fromtimestamp(times[0])

In [ ]:
newver = [ 'slide_windows_type' in par_cur for par_cur in pars]

In [ ]:
len(pars) - sum( newver )

In [ ]:
import matplotlib.pyplot as plt
COLOR = 'white'
mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR

plt.hist(days)

In [ ]:
len(srt)

In [ ]:
list ( f.keys() )

In [ ]:
f['par'][()]

In [ ]:
biotec falls,
new datacenter companies fall

In [ ]:
hpass

In [ ]:
len(  env2all_scores[env] )

In [ ]:
# (4, 20, 321) in orig;    so  vars x subj x times

In [ ]:
len(scores_cur_env_cur_subj)

In [ ]:
scs[1,...].shape

In [ ]:
scs.transpose( (2,0,1)).shape

In [ ]:
scs.shape

In [ ]:
from mne.stats import permutation_cluster_1samp_test
help(permutation_cluster_1samp_test)

In [ ]:
len(tminmaxs)

In [ ]:
env2all_scores[env].shape

In [ ]:
print( len(subjects), len(tmins), scs.shape )

In [ ]:
len(env2all_scores['stable'])

In [ ]:
len(env2all_scores['random'])

In [ ]:
scs.shape

In [ ]:
len(tmins)

In [ ]:
env2all_scores

In [ ]:
dd

In [ ]:
os.listdir('/p/project/icei-hbp-2020-0012/lyon/memerr/data2/sub20_YOGCJKKB/results')

In [ ]:
os.listdir(results_folder)

In [ ]:
import sys
#for subject in subjects:
#for subject in subjects[:1]:
control_type = 'target'  # 'movement', 'feedback', 'target' or 'belief'
control_type = 'movement'  # 'movement', 'feedback', 'target' or 'belief'
script_name = pjoin(scripts_dir,'td_long_fig2.py')
for hpass in ['no_filter','0.1','0.05']:
    for regression_type in ['Ridge', 'xgboost']:    
        ipy.run_line_magic('run', f'-i {script_name}')
        #sys.exit(1)
        #print('saved_successfully =', saved_successfully)

In [ ]:
import sys
#for subject in subjects:
#for subject in subjects[:1]:
control_type = 'target'  # 'movement', 'feedback', 'target' or 'belief'
control_type = 'movement'  # 'movement', 'feedback', 'target' or 'belief'
script_name = pjoin(scripts_dir,'td_baselined_final_fig2.py')
for hpass in ['no_filter','0.1','0.05']:
    for regression_type in ['Ridge', 'xgboost']:    
        ipy.run_line_magic('run', f'-i {script_name}')
        sys.exit(1)
        #print('saved_successfully =', saved_successfully)

In [ ]:
path_data

In [ ]:
import gc; gc.collect()

In [ ]:
fnames_missing

In [ ]:
fn = '/home/demitau/data_Quentin/full_experiments/data2/sub01_WGPOZPEE/results/spoc_home2_0.05/stable_xgboost_scores_prevmovement_preverrors_errors_prevbelief_gamma.npy'
f = np.load(fn)

In [ ]:
'sub01_WGPOZPEE spoc_home2_0.05 stable_xgboost_scores_prevmovement_preverrors_errors_prevbelief_gamma.npy',

In [ ]:
f

In [ ]:
fnames_zero_size

In [ ]:
fname_full

In [ ]:
all_scores_stable

In [ ]:
#for subject in subjects:
#for subject in subjects[:1]:
script_name = pjoin(scripts_dir,'td_long_fig2.py')
ipy.run_line_magic('run', f'-i {script_name}')